In [1]:
"""This is testing on real stars""";

In [2]:
"""
This program tests Affine+Ditrortion35 model
on artificially generated data (pairs of star coordinates)
""";

In [3]:
from pylab import *
from PIL import Image, ImageDraw
from functools import partial
import os

Get star pairs and calculate NUM_STAR_PAIRS (at least 5 because it's minimum needed for affine+distortion35 model)   

In [4]:
# whether to divide coordinates by SCALE_FAC 
# or not (zoomed coords or not)
zoomed_coords = True 
center_only = True # use only central stars
SCALE_FAC = 4.0 # Scale factor of coordinates

In [5]:
"""
Load star coords from txt-files
""";

In [6]:
# folder with coords files
coords_folder = 'data/star_coords/2016nov-11_txt/' 
images_folder = 'data/stars/2016nov-11/'

In [7]:
# 2016nov-11 jpg
fnames = [
    "20161122-191517-359.txt",
    "20161122-201517-375.txt",
    "20161122-211517-375.txt",
    "20161122-221517-375.txt"
]

In [8]:
date = fnames[0].split('.')[0]

In [9]:
im = Image.open(images_folder + "mod_" + date + "-1.jpg")
width, height = im.size
print("Image size:", width, height)

Image size: 3072 2304


In [10]:
xCenter = width // 2
yCenter = height // 2
CENTER_RAD = 700 # radius(px) of central part
print('CENTER_RAD:', CENTER_RAD)

CENTER_RAD: 700


In [11]:
coords_list = []
for fname in fnames:
    piece = np.loadtxt(coords_folder + os.sep + fname)
    coords_list.append(piece)

coords = np.vstack(coords_list)

In [12]:
if zoomed_coords:
    coords /= float(SCALE_FAC)
    coords = coords.round()
    print('Normal Star coordinates pairs (first 5):\n', coords[:5], '\n')


Normal Star coordinates pairs (first 5):
 [[ 1434.  1566.  1678.  1603.]
 [ 1622.  1681.  1867.  1716.]
 [ 1484.  1840.  1728.  1877.]
 [ 1552.  1994.  1798.  2032.]
 [ 2444.  2120.  2720.  2178.]] 



In [13]:
if center_only:
    coords_center = []
    
    for i in range(coords.shape[0]):
        _lx = coords[i, 0]
        _ly = coords[i, 1]
        _rx = coords[i, 2]
        _ry = coords[i, 3]
        if \
        (_lx - xCenter)**2 + (_ly - yCenter)**2 <= CENTER_RAD**2 and \
        (_rx - xCenter)**2 + (_ry - yCenter)**2 <= CENTER_RAD**2:
            coords_center.append(coords[i])
    
    coords = np.vstack(coords_center)
    
    print('Normal Star coordinates pairs in center:\n', coords[:5], '\n')

Normal Star coordinates pairs in center:
 [[ 1434.  1566.  1678.  1603.]
 [ 1622.  1681.  1867.  1716.]
 [ 1231.  1291.  1474.  1328.]
 [ 1438.  1360.  1680.  1397.]
 [ 1338.  1543.  1582.  1579.]] 



In [14]:
NUM_STAR_PAIRS = len(coords)
N = NUM_STAR_PAIRS
M = coords.shape[1] # {lX, lY, rX, rY} == 4
print('Number of Star coordinates pairs:', NUM_STAR_PAIRS)

Number of Star coordinates pairs: 15


In [15]:
leftX = coords[:, 0]
leftY = coords[:, 1]
rightX = coords[:, 2]
rightY = coords[:, 3]

print('''First 5 pairs
Left X: {}
Left Y: {}
'''.format(leftX[:5], leftY[:5])
)
print('''\
Right X: {}
Right Y: {}
'''.format(rightX[:5], rightY[:5])
)

First 5 pairs
Left X: [ 1434.  1622.  1231.  1438.  1338.]
Left Y: [ 1566.  1681.  1291.  1360.  1543.]

Right X: [ 1678.  1867.  1474.  1680.  1582.]
Right Y: [ 1603.  1716.  1328.  1397.  1579.]



In [16]:
ELL_RAD = 3

# Draw star pairs
scatterOriginal = Image.new('RGB', (width, height), 'lightgray')

draw = ImageDraw.Draw(scatterOriginal)

# Central point
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')

# Draw central part boundary
draw.ellipse((xCenter - CENTER_RAD, yCenter - CENTER_RAD, 
              xCenter + CENTER_RAD, yCenter + CENTER_RAD), outline='black')


for i in range(NUM_STAR_PAIRS): # draw star points
    draw.ellipse((leftX[i] - ELL_RAD, leftY[i] - ELL_RAD, 
                  leftX[i] + ELL_RAD, leftY[i] + ELL_RAD), fill='blue')


In [17]:
scatterOriginal.save('orig.png')

affine coeffincients  
(a,b,  
 c,d) -- for rotation matrix  
(e,f) -- for transition (shift)   

In [18]:
def affine_transform(xy, coeffs=(1,0,0,1,0,0)):
    assert coeffs != (1,0,0,1,0,0)
        
    _a, _b, _c, _d, _e, _f = coeffs
    x, y = xy
    return [
        _a * x + _b * y + _e,
        _c * x + _d * y + _f
    ]

In [19]:
# inputLeftX, inputLeftY, inputRightX, inputRightY are coordinates we get from measuring system
inputLeftX = leftX
inputLeftY = leftY

inputRightX = rightX
inputRightY = rightY

In [20]:
def correct_distort(xy, coeffs=(0,0)):
    assert coeffs != (0,0)
    
    # eps1, eps3 -- for left img
    # eps2, eps4 -- for right img
    _eps1_or_eps2, _eps3_or_eps4  = coeffs
    
    x, y = xy
    
    # squared distance from center to (x, y) point
    _r = (x - xCenter) ** 2 + (y - yCenter) ** 2
    
    return [
        x - (x - xCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 ),
        y - (y - yCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 )
    ]

### Test affine model

Calculate model coefficients

In [21]:
leftX = inputLeftX
leftY = inputLeftY
rightX = inputRightX
rightY = inputRightY

In [22]:
xi = np.zeros(2 * NUM_STAR_PAIRS)

for i in range(NUM_STAR_PAIRS): # fill the xi vector
    xi[2 * i] = rightX[i]
    xi[2 * i + 1] = rightY[i]

print('xi (first 5):\n', xi[:5])

xi (first 5):
 [ 1678.  1603.  1867.  1716.  1474.]


In [23]:
k = 6 # num of coeff-s

z = np.zeros(k)
arr = np.zeros((2 * NUM_STAR_PAIRS, k)) # matrix A

for i in range(NUM_STAR_PAIRS): # fill the A matrix
    
    arr[2 * i] = [leftX[i], leftY[i], 0, 0, 1, 0]

    arr[2 * i + 1] = [0, 0, leftX[i], leftY[i], 0, 1]

    
p_arr = pinv(arr, rcond=1e-20)
z = np.dot(p_arr, xi)

print("""
Affine coefficients:
%.4f %.4f %.4f %.4f 
%.2f %.2f""" % tuple(z))
print('cond(A): ', np.linalg.cond(arr))


Affine coefficients:
0.9955 0.0070 -0.0094 1.0013 
240.19 48.04
cond(A):  12204.1086347


In [24]:
"""
Align images and blend

a) Affine
""";

In [25]:
affine = partial(affine_transform, coeffs=tuple(z))

# Calc estimated (affine transformed) points
leftCoords = array(list(map(affine, zip(leftX, leftY))))

# Estimated coordinates
estLeftX = leftCoords[:, 0]
estLeftY = leftCoords[:, 1]


print('''Backward affine Left:
Left X: {}
Left Y: {}
'''.format(estLeftX[:5], estLeftY[:5])
)

print('''Right:
Right X: {}
Right Y: {}
'''.format(rightX[:5], rightY[:5])
)

Backward affine Left:
Left X: [ 1678.64587318  1866.59791155  1474.64289699  1681.1871164   1582.92018962]
Left Y: [ 1602.61997402  1716.0047944   1329.16733285  1396.3137712   1580.49133379]

Right:
Right X: [ 1678.  1867.  1474.  1680.  1582.]
Right Y: [ 1603.  1716.  1328.  1397.  1579.]



Calculate error metrics

1) $\Delta x_i, \Delta y_i, \; i = 1,N$

2) $\sigma^2 = \frac{1}{N} \sum\limits_{i=1}^{N} 
                \left( \Delta x_i^2 + \Delta y_i^2 \right)$

In [26]:
delX = abs(estLeftX - rightX)
delY = abs(estLeftY - rightY)
print("delX:", delX[:5])
print("delY:", delY[:5])

sigSqr = 1.0 / N * sum(delX**2 + delY**2)
mX = max(delX)
mY = max(delY)
m = max(mX, mY)

print("mX: %.4f mY: %.4f m: %.4f" % (mX, mY, m))
print("sigSqr: %.4f" % sigSqr)

delX: [ 0.64587318  0.40208845  0.64289699  1.1871164   0.92018962]
delY: [ 0.38002598  0.0047944   1.16733285  0.6862288   1.49133379]
mX: 4.0284 mY: 2.3902 m: 4.0284
sigSqr: 3.5958


Plot aligned star pairs

In [27]:
scatter = Image.new('RGB', (width, height), 'lightgray')


draw = ImageDraw.Draw(scatter)
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')


for i in range(NUM_STAR_PAIRS): # draw star points
    draw.ellipse((estLeftX[i] - ELL_RAD, estLeftY[i] - ELL_RAD, 
                  estLeftX[i] + ELL_RAD, estLeftY[i] + ELL_RAD), fill='red')
    
    draw.ellipse((rightX[i] - ELL_RAD, rightY[i] - ELL_RAD, 
                  rightX[i] + ELL_RAD, rightY[i] + ELL_RAD), fill='blue')

In [28]:
scatter.save('000.png')

### Test affine+distortion35 model

Calculate model coefficients

In [29]:
leftX = inputLeftX
leftY = inputLeftY
rightX = inputRightX
rightY = inputRightY

In [30]:
"""
c) Affine + Ditortion 3rd, 5th orders 
  (at least 5 stars)
""";

In [31]:
k35 = 10

z35 = np.zeros(k35)
arr35 = np.zeros((2 * N, k35)) # matrix A

for i in range(N): # fill the A matrix
    dist_l = (leftX[i] - xCenter) ** 2 + (leftY[i] - yCenter) ** 2
    dist_r = (rightX[i] - xCenter) ** 2 + (rightY[i] - yCenter) ** 2

    zx1 = (leftX[i] - xCenter) * dist_l
    zx2 = (rightX[i] - xCenter) * dist_r
    wx1 = (leftX[i] - xCenter) * dist_l ** 2
    wx2 = (rightX[i] - xCenter) * dist_r ** 2

    arr35[2 * i] = [leftX[i], leftY[i], 0, 0, 1, 0, -zx1, zx2, -wx1, wx2]

    zy1 = (leftY[i] - yCenter) * dist_l
    zy2 = (rightY[i] - yCenter) * dist_r
    wy1 = (leftY[i] - yCenter) * dist_l ** 2
    wy2 = (rightY[i] - yCenter) * dist_r ** 2

    arr35[2 * i + 1] = [0, 0, leftX[i], leftY[i], 0, 1, -zy1, zy2, -wy1, wy2]


In [32]:
p_arr35 = pinv(arr35, rcond=1e-20)
z35 = np.dot(p_arr35, xi)


print("""
Affine coefficients + Ditortion 3rd, 5th orders:

%.4f %.4f %.4f %.4f 
%.2f %.2f 
%.2e %.2e 
%.2e %.2e""" % tuple(z35))

print('cond(A): ', np.linalg.cond(arr35))


Affine coefficients + Ditortion 3rd, 5th orders:

0.9971 0.0086 -0.0102 1.0010 
232.84 49.22 
5.35e-08 3.72e-08 
-5.27e-14 -1.72e-14
cond(A):  1.90900558288e+15


In [33]:
"""
c) Affine + Ditortion3,5
""";

In [34]:
a = float(z35[0])
b = float(z35[1])
c = float(z35[2])
d = float(z35[3])
e = float(z35[4])
f = float(z35[5])

eps1 = float(z35[6])
eps2 = float(z35[7])
eps3 = float(z35[8])
eps4 = float(z35[9])

In [35]:
# Backward distort

correctDistortLeft = partial(correct_distort, coeffs=(eps1, eps3))
leftCoords = array(list(map(correctDistortLeft, zip(leftX, leftY))))
leftX = leftCoords[:, 0]
leftY = leftCoords[:, 1]


correctDistortRight = partial(correct_distort, coeffs=(eps2, eps4))
rightCoords = array(list(map(correctDistortRight, zip(rightX, rightY))))
estRightX35 = rightCoords[:, 0]
estRightY35 = rightCoords[:, 1]


# Backward affine
affine = partial(affine_transform, coeffs=(a,b,c,d,e,f))


leftCoords = array(list(map(affine, zip(leftX, leftY))))
estLeftX35 = leftCoords[:, 0]
estLeftY35 = leftCoords[:, 1]


print('''Backward distort+affine Left:
Left X: {}
Left Y: {}
'''.format(estLeftX35[:5], estLeftY35[:5])
)

print('''Backward distort Right:
Right X: {}
Right Y: {}
'''.format(estRightX35[:5], estRightY35[:5])
)

Backward distort+affine Left:
Left X: [ 1677.01519758  1863.68996342  1473.06242765  1678.70317163  1581.92469413]
Left Y: [ 1598.92687037  1709.62653379  1328.26610077  1395.43277037  1576.9179542 ]

Backward distort Right:
Right X: [ 1676.94227451  1862.78056405  1474.07893683  1679.58395972  1581.71160447]
Right Y: [ 1599.64060426  1708.81038708  1327.77592125  1396.2921537   1576.32293712]



Calculate error metrics

In [36]:
delX35 = abs(estLeftX35 - estRightX35)
delY35 = abs(estLeftY35 - estRightY35)
print("delX35:", delX35[:5])
print("delY35:", delY35[:5])

sigSqr35 = 1.0 / N * sum(delX35**2 + delY35**2)
mX35 = max(delX35)
mY35 = max(delY35)
m35 = max(mX35, mY35)

print("mX35: %.4f mY35: %.4f m35: %.4f" % (mX35, mY35, m35))
print("sigSqr35: %.4f" % sigSqr35)

delX35: [ 0.07292307  0.90939937  1.01650919  0.8807881   0.21308967]
delY35: [ 0.71373389  0.81614671  0.49017953  0.85938333  0.59501709]
mX35: 1.6837 mY35: 1.2512 m35: 1.6837
sigSqr35: 1.1172


Plot aligned star pairs

In [37]:
scatter35 = Image.new('RGB', (width, height), 'lightgray')


draw = ImageDraw.Draw(scatter35)
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')


for i in range(NUM_STAR_PAIRS): # draw star points
    draw.ellipse((estLeftX35[i] - ELL_RAD, estLeftY35[i] - ELL_RAD, 
                  estLeftX35[i] + ELL_RAD, estLeftY35[i] + ELL_RAD), fill='red')
    
    draw.ellipse((estRightX35[i] - ELL_RAD, estRightY35[i] - ELL_RAD, 
                  estRightX35[i] + ELL_RAD, estRightY35[i] + ELL_RAD), fill='blue')


In [38]:
scatter35.save('035.png')